#### Installation

* Mysql repository (dpkg) installation

```console
sudo dpkg -i mysql.deb
```

* Appropriate version of the package should be selected

```console
sudo apt install mysql-server mysql-shell
```

#### Setup

```console
sudo nano /etc/mysql/my.cnf
```

```
[client]
port = <port>
host = <#>
```

```console
systemctl restart mysql
```

#### Startup

```console
mysql
```

* Access denied for the current user

```console
sudo mysql
```

* Login with root without password

```console
mysql -h <hostname> -p <port> -u <user> -p
```

#### Create an User

```sql
alter user 'root'@'localhost' identified with caching_sha2_password by '<pwd>'
```

* identified with (caching_sha2_password, mysql_native_password)
* some backends require mysql_native_password

```sql
create user '<user-to-create>'@'localhost' identified with caching_sha2_password by '<pwd>'
``` 

```sql
drop user '<user-to-remove>'@'<host>'
```

```sql
flush privileges
```

* Semicolons are intentionally omitted

#### Give Privileges (User Scopes)

```sql
show grants for '<user>'@'<host>'
```

```sql
grant <privileges> on <database>.<table> to '<user>'@'<host>'
```

```sql
grant all privileges on *.* to '<user>'@'<host>' with grant option
```

* flush privileges is unnecessary

```sql
revoke <privileges> on <database>.<table> from '<user>'@'<host>'
```

#### Enable Loading Local Infile

```sql
show global variables like 'local_infile'
```

* local_infile == off

```sql
set global local_infile = 'ON'
```

#### Load Database from Local Infile

```console
mysqlsh -u 'yuninze' -p
```

```javascript
util.loadDump('airport-db',{threads:4, deferTableIndexes:'all', ignoreVersion:true})
```

```sql
load data infile '<.csv>'
into table <table>
fields terminated by ','
enclosed by ''
lines terminated by '\n'
ignore 1 lines
```

#### Get Info

```sql
show databases
```

```sql
show tables from <database>
```

```sql
show columns from <table>
```

#### Clauses

##### Unique Values by Column

```sql
select distinct <table.column> from <table> ...
```

##### Count Rows

```sql
select count(*) as nrows from <table0> union all
select count(*) from <table1> union all
select count(*) from <table2> order by row_n_table0
```

##### Group By Agg Count

```sql
select <table.column>, <aggfunc> from <table> where <table.column> <cond>
group by <table.column> order by <table.column> limit 10
```

##### Group By Agg Count Mean

```sql
select <table.column.categorical>, avg(<table.column.contiguous>) as avg_per_category, count(*) as count_per_category from <table>
where <table0.id>=<table1.id> and <cond>
order by <table.column.categorical> limit 10
```

##### Group By Agg Count Diff

```sql
select <table.column.categorical>, avg(datediff(flight_start,flight_end)) as avg_flight_time,
count(*) as count_flight from <table>
where <table0.id>=<table1.id> and <cond> order by <table.column.categorical> limit 10
```

##### Partial String Match

```sql
select lowered from (
  select lower(<table.column>) as lowered from <table>
)
where <cond> and <table.column> like 'ec%';
```

* Character match _
* Remainder match %
* '%[o-v]' or set '%[abcdef]' 등 레젝스 유사표현도 like로 사용 가능

##### Subquery

```sql
select <column.x> from <table.a>
where exists (
  select <column.y> from <table.b>
  where <table.b.column.q> = <table.a.column.w> and <column.z> <cond>
);
```

```sql
select <name> from <product> where <id> = any(select <id> from <order> where <quant> = 10)
```

##### Isna-Isblank Equivalent

```sql
select <name>, <price> * (<quant> + ifnull[coalesce](<qaunt_coming_product>, 0)) from product
```

##### uint and zerofill

```sql
decimal(5,3) zerofill
```

* decimal[numeric](<p, precision>, <d, number of digits>) [<zerofill, padding>, <unsigned, no negative values>]

##### select distinct statements work like unique method

```sql
select distinct q,w,e from r
where q + w + e > const;
```

```sql
select q,w,e from r
where q + w + e > const
group by q, w, e;
```

* distinct은 전형적인 unique 메서드임
* 전자는 q, w, e 각각의, 후자는 (q, w, e)에 대해 unique array을 냄

##### Exist statements take boolean(tinyint(1)) arrays as condition

```sql
<outer_expr> in (
  select <inner_expr> from ... where <subquery_where>
)
```

```sql
exists (
  select 1 from ... where <subquery_where> and <outer_expr> = <inner_expr>
)
```

* Later is better

##### Using exist statements provide performance efficiency resemble to boolean indexing

```sql
select class_num, class_name from class
where exists (
  select * from roaster where class.class_num = roaster.class_num
)
```
